# Classical Component VQE

In [1]:
import qsharp
# qsharp.reload() helped when couldn't locate any namespaces
# qsharp.get_available_operations_by_namespace() was good for debugging

Preparing Q# environment...


In [2]:
qsharp.get_available_operations_by_namespace()

{'SLAP': ['Ansatz', 'GetEnergyByTrotterization', 'VQEGetEnergy']}

In [3]:
qsharp.packages.add("Microsoft.Quantum.Chemistry")
# packs = qsharp.Packages(qsharp.client)
# packs.add(package_name="Microsoft.Quantum.Chemistry")

Adding package Microsoft.Quantum.Chemistry.

In [4]:
qsharp.reload()
qsharp.get_available_operations_by_namespace()

Reloading workspace.

{'SLAP': ['Ansatz', 'GetEnergyByTrotterization', 'VQEGetEnergy']}

In [5]:
from SLAP import VQEGetEnergy, GetEnergyByTrotterization

In [6]:
# from qdk.chemistry import Molecule
from qdk.chemistry.broombridge import load_and_encode

Adding package microsoft.quantum.chemistry.jupyter.

In [7]:
# caffine = Molecule.from_xyz("data/caffeine/caffeine.xyz")

In [8]:
# caffine.mol

In [9]:
from scipy.optimize import minimize

def VQEGetEnergyWrapper(var_params, molecule_data, num_samples) -> float:
    theta1, theta2, theta3 = var_params
    return VQEGetEnergy.simulate(
        JWEncodedData=molecule_data,
        theta1=theta1,
        theta2=theta2,
        theta3=theta3,
        nSamples=num_samples
    )

def VQE_Classical_Routine(molecule_data, initial_var_params, num_samples):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """
    opt_result = minimize(
        VQEGetEnergyWrapper,
        initial_var_params,
        args=(molecule_data, num_samples,),
        method="COBYLA",
        tol=0.000001,
        options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05}
    )

    if opt_result.success:
        print(opt_result.message)
        print(f"Lowest Energy: {opt_result.fun} Ha")
        print(f"Number of evaluations: {opt_result.nfev}")
        print(f"Optimal parameters found: {opt_result.x}")

    return opt_result

In [17]:
encoded_data = load_and_encode("data/caffeine/caffeine.yaml")

In [11]:
encoded_data

(4,
 ([([0], [0.1709946636999999]),
   ([1], [0.1709946636999999]),
   ([2], [-0.22217713245000004]),
   ([3], [-0.22217713245000004])],
  [([0, 1], [0.168559024425]),
   ([0, 2], [0.1204687916]),
   ([0, 3], [0.165809533925]),
   ([1, 2], [0.165809533925]),
   ([1, 3], [0.1204687916]),
   ([2, 3], [0.174287548625])],
  [],
  [([0, 1, 2, 3], [0.0, -0.045340742325, 0.0, 0.045340742325])]),
 (2,
  [((0.993584134540649, 0.0), [0, 1]), ((-0.11309539154673656, 0.0), [2, 3])]),
 -0.10055676285765724)

In [18]:
VQE_Classical_Routine(encoded_data, [0.001, -0.001, 0.001], 1)

In [13]:
VQEGetEnergy.simulate(
    JWEncodedData=encoded_data,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    nSamples=10000
)

-1.116610433157657

In [14]:
# VQEGetEnergy.estimate_resources(
#     JWEncodedData=encoded_data,
#     theta1=0.001,
#     theta2=-0.001,
#     theta3=0.001,
#     nSamples=1
# )

In [15]:
GetEnergyByTrotterization.simulate(
    qSharpData=encoded_data,
    nBitsPrecision=10,
    trotterStepSize=0.2,
    trotterOrder=1
)

(-0.20708740636456158, -1.135993794680465)

In [16]:
# GetEnergyByTrotterization.estimate_resources(
#     qSharpData=encoded_data,
#     nBitsPrecision=10,
#     trotterStepSize=0.2,
#     trotterOrder=1
# )